In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path_to_your_dir = 'drive/My Drive/PlanetaSimillarFilms/'

from keras.layers import Input, Embedding, Dot, Reshape, Dense
from keras.models import Model
from keras.models import load_model
from tqdm import tqdm_notebook as tqdm

import pandas as pd
import numpy as np
import random
import requests
import itertools as it

random.seed(100)
np.set_printoptions(suppress=True)
pd.options.display.max_columns = 999
pd.options.display.max_rows = 200


In [0]:
def make_pairs_for_train_NN(df):
  
  
  st_wrds_not_stemmed = ["mightn't", 'you', 'theirs', 'aren', 'has', 'she', 'they', 'over',
       'below', 'how', 'yours', "hasn't", 'here', 'where', 'nor',
       'couldn', "you've", 'wouldn', 'yourselves', 'if', 'shouldn', 's',
       'before', 'as', 'between', 'mustn', 'doesn', 'hadn', 'too', 'won',
       'their', 'which', 'few', "you're", 'what', 'with', 'and', 'itself',
       'we', 'do', 'until', 'up', 'then', 'y', "couldn't", 'when', 'into',
       'themselves', "don't", 'hasn', 'her', 'its', 're', 'does', 'an',
       'mightn', "weren't", 'had', 'haven', 'doing', 'shan', 'i', "you'd",
       'ours', 'there', 'was', 'some', 'needn', 'he', 'hers', 'on', 'why',
       "should've", "she's", 'own', "hadn't", 'weren', 'of', 'me',
       'while', "won't", "you'll", 'these', "needn't", 'now', 'once',
       "mustn't", 'll', 'only', 'that', "it's", 'didn', 'no', 'ain',
       'him', 'each', 'to', 'during', 'same', 'other', 'are', 'from', 'a',
       'don', 'all', 'who', "shouldn't", "wasn't", "wouldn't", 'his',
       'have', 't', 'wasn', 'under', 'about', 'be', 'or', 'for', 'my',
       'so', 'through', 'those', 'not', 'isn', 'in', 'herself', 'any',
       'yourself', "isn't", "aren't", 'very', 'against', 'again',
       "didn't", 'after', 'them', 'whom', 'the', 'down', "that'll", 'out',
       'ourselves', "doesn't", "shan't", 'but', 'should', 'just', 'ma',
       'being', 'because', 'at', 'myself', 'himself', 'above', 'more',
       'been', 'by', 'further', 've', 'm', 'most', 'off', 'it', 'than',
       'were', 'having', 'such', 'your', 'will', 'o', 'our', 'can',
       'this', 'd', 'is', 'did', 'am', "haven't", 'both', '']
  
  
  
  
  
  st_wrds = ["mightn't", 'you', 'their', 'aren', 'has', 'she', 'they', 'over', 'below', 'how', 'your', "hasn't",
             'here', 'where', 'nor', 'couldn', "you'v", 'wouldn', 'yourselv', 'if', 'shouldn', 's', 'befor', 'as',
             'between', 'mustn', 'doesn', 'hadn', 'too', 'won', 'their', 'which', 'few', "you'r", 'what', 'with', 'and',
             'itself', 'we', 'do', 'until', 'up', 'then', 'y', "couldn't", 'when', 'into', 'themselv', "don't", 'hasn', 'her',
             'it', 're', 'doe', 'an', 'mightn', "weren't", 'had', 'haven', 'do', 'shan', 'i', "you'd", 'our', 'there', 'was', 'some',
             'needn', 'he', 'her', 'on', 'whi', "should'v", 'she', 'own', "hadn't", 'weren', 'of', 'me', 'while', "won't", "you'll",
             'these', "needn't", 'now', 'onc', "mustn't", 'll', 'onli', 'that', 'it', 'didn', 'no', 'ain', 'him', 'each', 'to',
             'dure', 'same', 'other', 'are', 'from', 'a', 'don', 'all', 'who', "shouldn't", "wasn't", "wouldn't", 'his', 'have', 't',
             'wasn', 'under', 'about', 'be', 'or', 'for', 'my', 'so', 'through', 'those', 'not', 'isn', 'in', 'herself', 'ani', 'yourself',
             "isn't", "aren't", 'veri', 'against', 'again', "didn't", 'after', 'them', 'whom', 'the', 'down', "that'll", 'out', 'ourselv',
             "doesn't", "shan't", 'but', 'should', 'just', 'ma', 'be', 'becaus', 'at', 'myself', 'himself', 'abov', 'more', 'been', 'by',
             'further', 've', 'm', 'most', 'off', 'it', 'than', 'were', 'have', 'such', 'your', 'will', 'o',
             'our', 'can', 'this', 'd', 'is', 'did', 'am', "haven't", 'both', '']
  
  
  flat_list_all_tags = [item for sublist in df.bag_of_words_for_train_nn.apply(lambda x: x.split(' ')) for item in sublist]
  
  st_wrds.extend(pd.Series(flat_list_all_tags).value_counts()[pd.Series(flat_list_all_tags).value_counts() == 1].index.tolist())
  
    
  df.bag_of_words_for_train_nn = df.bag_of_words_for_train_nn.apply(lambda x: [i for i in x.split(' ') if i not in st_wrds])
  
  film_index = {film: idx for idx, film in enumerate(df.imdbID.values)}
  index_film = {idx: film for film, idx in film_index.items()}
  
  list_of_lists = df.bag_of_words_for_train_nn.values
  flat_list = [item for sublist in list_of_lists for item in sublist]
  tags = pd.Series(flat_list).value_counts().to_dict()
  tag_index = {tag: idx for idx, tag in enumerate(tags)}
  index_tag = {idx: tag for tag, idx in tag_index.items()}
  
  pairs = []
  films = df.imdbID.values

  # Iterate through each book
  for film in films:
      # Iterate through the links in the book
      pairs.extend((film_index[film], tag_index[tag.lower()]) for tag in df[df.imdbID == film].bag_of_words_for_train_nn.values[0] if tag.lower() in tags)
      
  pairs_set = set(pairs)

  
  return pairs, pairs_set, tag_index, film_index, index_film, tags, films

In [0]:
def generate_batch(pairs, pairs_set, films, tags, n_positive = 50, negative_ratio = 1.0):
    """Generate batches of samples for training"""
    batch_size = n_positive * (1 + negative_ratio)
    batch = np.zeros((batch_size, 3))
    

    neg_label = -1
    
    # This creates a generator
    while True:
        # randomly choose positive examples
        for idx, (film_id, tag_id) in enumerate(random.sample(pairs, n_positive)):
            batch[idx, :] = (film_id, tag_id, 1)

        # Increment idx by 1
        idx += 1
        
        # Add negative examples until reach batch size
        while idx < batch_size:
            
            # random selection
            random_film = random.randrange(len(films))
            random_tag = random.randrange(len(tags))
            
            # Check to make sure this is not a positive example
            if (random_film, random_tag) not in pairs_set:
                
                # Add to batch and increment index
                batch[idx, :] = (random_film, random_tag, neg_label)
                idx += 1
                
        # Make sure to shuffle order
        np.random.shuffle(batch)
        yield {'film': batch[:, 0], 'tag': batch[:, 1]}, batch[:, 2]

In [0]:
def film_embedding_model(film_index, tag_index, embedding_size = 50):
    """Model to embed books and wikilinks using the functional API.
       Trained to discern if a link is present in a article"""
    
    # Both inputs are 1-dimensional
    film = Input(name = 'film', shape = [1])
    tag = Input(name = 'tag', shape = [1])
    
    
    # Embedding the book (shape will be (None, 1, 50))
    film_embedding = Embedding(name = 'film_embedding',
                               input_dim = len(film_index),
                               output_dim = embedding_size)(film)
    
    # Embedding the link (shape will be (None, 1, 50))
    tag_embedding = Embedding(name = 'tag_embedding',
                               input_dim = len(tag_index),
                               output_dim = embedding_size)(tag)
    
    # Merge the layers with a dot product along the second axis (shape will be (None, 1, 1))
    merged = Dot(name = 'dot_product', normalize = True, axes = 2)([film_embedding, tag_embedding])
    
    # Reshape to be a single number (shape will be (None, 1))
    merged = Reshape(target_shape = [1])(merged)


    model = Model(inputs = [film, tag], outputs = merged)
    model.compile(optimizer = 'Adam', loss = 'mse')
    
    return model


In [0]:
def get_similar_film_by_nn(name, model, film_index, index_film):
  
  film_layer = model.get_layer('film_embedding')
  weights = film_layer.get_weights()[0]
  weights = weights / np.linalg.norm(weights, axis = 1).reshape((-1, 1))

  index = film_index
  rindex = index_film

  # Check to make sure `name` is in index
  try:
      # Calculate dot product between book and all others
      dists = np.dot(weights, weights[index[name]])
  except KeyError:
      print(f'{name} Not Found.')
      return

  # Sort distance indexes from smallest to largest
  sorted_dists = np.argsort(dists)

  # Take the last n sorted distances
  closest = sorted_dists[::-1][1:]
#   closest = sorted_dists[:]
  
  return closest, dists[sorted_dists[::-1][1:]]

In [0]:
def train_model(df, pairs, pairs_set, tag_index, film_index, index_film, tags, films, epoch, batch_size):
  
   
  model = film_embedding_model(film_index, tag_index)
  
  gen = generate_batch(pairs, pairs_set, films, tags, batch_size, negative_ratio = 2)
  
  model.fit_generator(gen, epochs = epoch, 
                      steps_per_epoch = len(pairs) // batch_size,
                      verbose = 1)
  
  return model


In [0]:
def get_chunk(df, IMDB_ID, closest_films, film_index):

  film_for_reccomendations = df[df.imdbID == IMDB_ID]
  df = df[df.imdbID != IMDB_ID]

  current_rated = film_for_reccomendations.rated.values[0]
  current_country = film_for_reccomendations.country.values[0]
  current_directors = film_for_reccomendations.director.values[0]
  current_genre = film_for_reccomendations.genre.values[0].split(' ')
  current_keywords_whole = film_for_reccomendations.keywords_whole.values[0]
  current_keywords_by_space = film_for_reccomendations.keywords_by_space.values[0].split(' ')
  current_bag_of_words_for_train_nn = film_for_reccomendations.soup.values[0].split(' ')

  chunk = []
  
    ######################### DIRECTOR SORT #########################

  reccomendation_by_director = []

  director_list = ['christopher_nolan', 'wes_anderson', 'quentin_tarantino', 'woody_allen', 'james_cameron', 'david_fincher', 'guy_ritchie',
                  'george_lucas', 'lana_wachowski', 'david_lynch', 'hayao_miyazaki', 'luc_besson', 'pedro_almodóvar',
                  'martin_scorsese', 'francis_ford_coppola', 'clint_eastwood', 'peter_jackson', 'tim_burton', 'joel_coen',
                  'steven_soderbergh', 'kar-wai_wong', 'robert_zemeckis', 'steven_spielberg', 'roman_polanski', 'aamir_khan', 'jean-pierre_jeunet',
                  'gus_van_sant', 'thomas_vinterberg', 'asghar_farhadi', 'denis_villeneuve', 'guillermo_del_toro', 'alfonso_cuarón',
                  'darren_aronofsky', 'makoto_shinkai', 'chan-wook_park', 'richard_linklater', 'martin_mcdonagh', 'gaspar_noé',
                  'lars_von_trier', 'sofia_coppola', 'alejandro_g._iñárritu']

  for director in director_list:
      if director in current_directors:
          director_df = df[df.director.str.contains(director)]
          if not director_df.empty:
              reccomendation_by_director.extend(director_df.imdbID.tolist())
              reccomendation_by_director_index = [film_index[i] for i in reccomendation_by_director]
              for i in closest_films:
                if i in reccomendation_by_director_index:
                  chunk.append([i, ' director ' + director])


  ######################### AGE SORT #########################
  
  rated_pg_13 = df[df.rated == 'PG-13']
  
  if 'G' == current_rated or 'PG' == current_rated:
    rated_df = df[np.logical_or(df.rated == 'G', df.rated == 'PG')]
    age_limit = 'G or PG'
#     print('G and PG')
  else:
    rated_df = df[~np.logical_or(df.rated == 'G', df.rated == 'PG')]
    age_limit = 'PG-13 or R'
#     print('PG-13 or R')



                  


  country_ukraine_russia_df = rated_df[np.logical_or(rated_df.country.str.contains('ukraine'), rated_df.country.str.contains('russia'))]
  country_usa_df = rated_df[rated_df.country.str.contains('usa')]
  country_others_df = rated_df[~np.logical_or(np.logical_or(rated_df.country.str.contains('ukraine'), rated_df.country.str.contains('russia')),
                                            rated_df.country.str.contains('usa') )]

  country_not_ukr_rus_df = rated_df[~np.logical_or(rated_df.country.str.contains('ukraine'), rated_df.country.str.contains('russia'))]  
  
  
  
  hard_hierarchy = {
      'group_GENRE_animation_and_TAG_minion':{
          'type':'Soup_just_1',
          'soup':['genre_animation', 'minion'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_animation_and_TAG_racing':{
          'type':'Soup_just_1',
          'soup':['genre_animation', 'racing'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_animation_and_TAG_superhero':{
          'type':'Soup_just_1',
          'soup':['genre_animation', 'superhero'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_animation_and_TAG_disney':{
          'type':'Soup_just_1',
          'soup':['genre_animation', 'disney'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_animation_AND_musical_OR_animation_AND_music':{
          'type':'Soup_OR_2x2',
          'soup':[['genre_animation', 'genre_musical'],
                  ['genre_animation', 'genre_music']],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_animation_AND_TAG_dragon':{
          'type':'Soup_just_1',
          'soup':['genre_animation', 'dragon'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_animation_AND_comedy':{
          'type':'Soup_just_1',
          'soup':['genre_animation','genre_comedy'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_animation':{
          'type':'Soup_just_1',
          'soup':['genre_animation'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_TAG_marvel_comic':{
          'type':'Soup_just_1',
          'soup':['marvel_comics'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_TAG_dc_comic':{
          'type':'Soup_just_1',
          'soup':['dc_comics'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_TAG_based_on_comic_book':{
          'type':'Soup_just_1',
          'soup':['based_on_comic_book'],
          'skip_country': False,
          'include_in_next': False
      },
      
      'group_GENRE_musical_and_TAG_dancing':{
          'type':'Soup_just_1',
          'soup':['genre_musical', 'dancing'],
          'skip_country': True,
          'include_in_next': False
      },
      'group_GENRE_musical':{
          'type':'Soup_just_1',
          'soup':['genre_musical'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_family_AND_TAG_animal':{
          'type':'Soup_just_1_notEXACTLY_just_1',
          'soup':['genre_family'],
          'not_exactly_tag':['animal'],
          'times':3,
          'skip_country': True,
          'include_in_next': True
      },
      'group_GENRE_drama_AND_family':{
          'type':'Soup_just_1',
          'soup':['genre_drama', 'genre_family'],
          'skip_country': False,
          'include_in_next': False
      },
      
      'group_GENRE_music_TAG_dancing':{
          'type':'Soup_just_1',
          'soup':['genre_music', 'dancing'],
          'skip_country': True,
          'include_in_next': False
      },
      'group_GENRE_biography_music_OR_history_music_OR_drama_music':{
          'type':'Soup_OR_3x2',
          'soup':[['genre_biography', 'genre_music'],
                  ['genre_history', 'genre_music'],
                  ['genre_drama', 'genre_music']],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_music':{
          'type':'Soup_just_1',
          'soup':['genre_music'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_documentary_war_OR_history_war_OR_biography_war':{
          'type':'Soup_OR_3x2',
          'soup':[['genre_documentary', 'genre_war'],
                  ['genre_history', 'genre_war'],
                  ['genre_biography', 'genre_war']],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_drama_AND_war':{
          'type':'Soup_just_1',
          'soup':['genre_drama', 'genre_war'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_documentary_OR_history_OR_biography_AND_TAG_world_war_two':{
          'type':'Soup_OR_3x2',
          'soup':[['genre_documentary', 'world_war_two'],
                  ['genre_history', 'world_war_two'],
                  ['genre_biography', 'world_war_two']],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_crime_AND_TAG_gangster':{
          'type':'Soup_just_1',
          'soup':['genre_crime', 'gangster'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_sport_AND_comedy':{
          'type':'Soup_just_1',
          'soup':['genre_sport', 'genre_comedy'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_sport':{
          'type':'Soup_just_1',
          'soup':['genre_sport'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama':{
          'type':'Soup_OR_3x2',
          'soup':[['genre_biography', 'genre_drama'],
                  ['genre_history', 'genre_drama'],
                  ['genre_documentary', 'genre_drama']],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_western':{
          'type':'Soup_just_1',
          'soup':['genre_western'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_comedy_AND_horror':{
          'type':'Soup_just_1',
          'soup':['genre_comedy', 'genre_horror'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_comedy_AND_TAG_zombie':{
          'type':'Soup_just_1',
          'soup':['genre_comedy', 'zombie'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_comedy_AND_TAG_kitchen':{
          'type':'Soup_just_1',
          'soup':['genre_comedy', 'kitchen'],
          'skip_country': False,
          'include_in_next': True
      },
      
      'group_GENRE_comedy_and_TAGS_love_relationships':{
          'type':'genre_tags_or_11',
          'soup':[['genre_comedy', 'husband_wife_relationship'],
                 ['genre_comedy', 'boyfriend_girlfriend_relationship'],
                 ['genre_comedy', 'ex_husband_ex_wife_relationship'],
                 ['genre_comedy', 'interracial_relationship'],
                 ['genre_comedy', 'older_man_younger_woman_relationship'],
                 ['genre_comedy', 'ex_boyfriend_ex_girlfriend_relationship'],
                 ['genre_comedy', 'older_woman_younger_man_relationship'],
                 ['genre_comedy', 'male_female_relationship'],
                 ['genre_comedy', 'fiance_fiancee_relationship'],
                 ['genre_comedy', 'ex_girlfriend_ex_boyfriend_relationship'],
                 ['genre_comedy', 'girlfriend_boyfriend_relationship']],
          'skip_country': False,
          'include_in_next': True
      },
      
      'group_GENRE_comedy_and_TAGS_family_relationships':{
          'type':'genre_tags_or_21',
          'soup':[['genre_comedy', 'father_son_relationship'],
                 ['genre_comedy', 'father_daughter_relationship'],
                 ['genre_comedy', 'mother_son_relationship'],
                 ['genre_comedy', 'mother_daughter_relationship'],
                 ['genre_comedy', 'brother_sister_relationship'],
                 ['genre_comedy', 'brother_brother_relationship'],
                 ['genre_comedy', 'family_relationships'],
                 ['genre_comedy', 'sister_sister_relationship'],
                 ['genre_comedy', 'aunt_niece_relationship'],
                 ['genre_comedy', 'uncle_nephew_relationship'],
                 ['genre_comedy', 'grandfather_grandson_relationship'],
                 ['genre_comedy', 'cousin_cousin_relationship'],
                 ['genre_comedy', 'grandmother_granddaughter_relationship'],
                 ['genre_comedy', 'aunt_nephew_relationship'],
                 ['genre_comedy', 'uncle_niece_relationship'],
                 ['genre_comedy', 'grandfather_granddaughter_relationship'],
                 ['genre_comedy', 'grandmother_grandson_relationship'],
                 ['genre_comedy', 'stepfather_stepdaughter_relationship'],
                 ['genre_comedy', 'father_in_law_daughter_in_law_relationship'],
                 ['genre_comedy', 'mother_in_law_daughter_in_law_relationship'],
                 ['genre_comedy', 'father_in_law_son_in_law_relationship']],
          'skip_country': False,
          'include_in_next': True
      },
      
      'group_GENRE_comedy_AND_drama':{
          'type':'Soup_just_1',
          'soup':['genre_comedy', 'genre_drama'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_comedy_AND_TAG_sex':{
          'type':'Soup_just_1_notEXACTLY_just_1',
          'soup':['genre_comedy'],
          'not_exactly_tag':['sex'],
          'times':3,
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_comedy':{
          'type':'Soup_just_1',
          'soup':['genre_comedy'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_horror_OR_drama_AND_TAG_post_apocalypse':{
          'type':'Soup_OR_2x2',
          'soup':[['genre_horror', 'post_apocalypse'],
                  ['genre_drama', 'post_apocalypse']],
          'skip_country': False,
          'include_in_next': False
      },

      'group_TAG_zombie':{
          'type':'Soup_just_1',
          'soup':['zombie'],
          'skip_country': False,
          'include_in_next': False
      },
      
      'group_GENRE_horror_AND_TAG_time_loop':{
          'type':'Soup_just_1',
          'soup':['genre_horror', 'time_loop'],
          'skip_country': False,
          'include_in_next': False
      },
      
      
      'group_GENRE_horror_AND_mystery_AND_thriller':{
          'type':'Soup_just_1',
          'soup':['genre_horror', 'genre_mystery', 'genre_thriller'],
          'skip_country': False,
          'include_in_next': True
      },
      
      'group_GENRE_drama_OR_thriller_AND_TAG_superhero':{
          'type':'Soup_OR_2x2',
          'soup':[['genre_thriller', 'superhero'],
                  ['genre_drama', 'superhero']],
          'skip_country': False,
          'include_in_next': False
      },
      
      'group_GENRE_scifi_AND_TAG_female_protagonist':{
          'type':'Soup_just_1',
          'soup':['genre_sci-fi', 'female_protagonist'],
          'skip_country': False,
          'include_in_next': False
      },
      
      'group_GENRE_action_AND_TAG_female_protagonist':{
          'type':'Soup_just_1',
          'soup':['genre_action', 'female_protagonist'],
          'skip_country': False,
          'include_in_next': False
      },
      
      'group_GENRE_horror_AND_TAG_shark':{
          'type':'Soup_just_1_notEXACTLY_just_1',
          'soup':['genre_horror'],
          'not_exactly_tag':['shark'],
          'times':3,
          'skip_country': True,
          'include_in_next': True
      },
            
      'group_GENRE_scifi_AND_horror':{
          'type':'Soup_just_1',
          'soup':['genre_sci-fi', 'genre_horror'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_sci-fi_AND_TAG_space_travel_OR_space':{
          'type':'Soup_OR_2x2',
          'soup':[['genre_sci-fi', 'space_travel'],
                  ['genre_sci-fi', 'space']],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_sci-fi_AND_TAG_martial_arts':{
          'type':'Soup_just_1',
          'soup':['genre_sci-fi', 'martial_arts'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_sci-fi_AND_TAG_survivor':{
          'type':'Soup_just_1',
          'soup':['genre_sci-fi', 'survivor'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_thriller_AND_horror':{
          'type':'Soup_just_1',
          'soup':['genre_thriller', 'genre_horror'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_action_AND_thriller':{
          'type':'Soup_just_1',
          'soup':['genre_action', 'genre_thriller'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_adventure_AND_family_AND_fantasy':{
          'type':'Soup_just_1',
          'soup':['genre_adventure', 'genre_family', 'genre_fantasy'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_scifi':{
          'type':'Soup_just_1',
          'soup':['genre_sci-fi'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_biography':{
          'type':'Soup_just_1',
          'soup':['genre_biography'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_history':{
          'type':'Soup_just_1',
          'soup':['genre_history'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_documentary':{
          'type':'Soup_just_1',
          'soup':['genre_documentary'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_alien':{
          'type':'Soup_just_1',
          'soup':['alien'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_drama_and_TAGS_love_relationships':{
          'type':'genre_tags_or_11',
          'soup':[['genre_comedy', 'husband_wife_relationship'],
                 ['genre_comedy', 'boyfriend_girlfriend_relationship'],
                 ['genre_comedy', 'ex_husband_ex_wife_relationship'],
                 ['genre_comedy', 'interracial_relationship'],
                 ['genre_comedy', 'older_man_younger_woman_relationship'],
                 ['genre_comedy', 'ex_boyfriend_ex_girlfriend_relationship'],
                 ['genre_comedy', 'older_woman_younger_man_relationship'],
                 ['genre_comedy', 'male_female_relationship'],
                 ['genre_comedy', 'fiance_fiancee_relationship'],
                 ['genre_comedy', 'ex_girlfriend_ex_boyfriend_relationship'],
                 ['genre_comedy', 'girlfriend_boyfriend_relationship']],
          'skip_country': False,
          'include_in_next': True
      },
      
      'group_GENRE_drama_and_TAGS_family_relationships':{
          'type':'genre_tags_or_21',
          'soup':[['genre_drama', 'father_son_relationship'],
                 ['genre_drama', 'father_daughter_relationship'],
                 ['genre_drama', 'mother_son_relationship'],
                 ['genre_drama', 'mother_daughter_relationship'],
                 ['genre_drama', 'brother_sister_relationship'],
                 ['genre_drama', 'brother_brother_relationship'],
                 ['genre_drama', 'family_relationships'],
                 ['genre_drama', 'sister_sister_relationship'],
                 ['genre_drama', 'aunt_niece_relationship'],
                 ['genre_drama', 'uncle_nephew_relationship'],
                 ['genre_drama', 'grandfather_grandson_relationship'],
                 ['genre_drama', 'cousin_cousin_relationship'],
                 ['genre_drama', 'grandmother_granddaughter_relationship'],
                 ['genre_drama', 'aunt_nephew_relationship'],
                 ['genre_drama', 'uncle_niece_relationship'],
                 ['genre_drama', 'grandfather_granddaughter_relationship'],
                 ['genre_drama', 'grandmother_grandson_relationship'],
                 ['genre_drama', 'stepfather_stepdaughter_relationship'],
                 ['genre_drama', 'father_in_law_daughter_in_law_relationship'],
                 ['genre_drama', 'mother_in_law_daughter_in_law_relationship'],
                 ['genre_drama', 'father_in_law_son_in_law_relationship']],
          'skip_country': False,
          'include_in_next': True
      },
      
      'group_GENRE_drama_AND_romance':{
          'type':'Soup_just_1',
          'soup':['genre_drama', 'genre_romance'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_drama_AND_thriller':{
          'type':'Soup_just_1',
          'soup':['genre_drama', 'genre_thriller'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_fantasy_AND_adventure':{
          'type':'Soup_just_1',
          'soup':['genre_fantasy', 'genre_adventure'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_fantasy':{
          'type':'Soup_just_1',
          'soup':['genre_fantasy'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_action':{
          'type':'Soup_just_1',
          'soup':['genre_action'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_adventure':{
          'type':'Soup_just_1',
          'soup':['genre_adventure'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_horror':{
          'type':'Soup_just_1',
          'soup':['genre_horror'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_thriller':{
          'type':'Soup_just_1',
          'soup':['genre_thriller'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_drama':{
          'type':'Soup_just_1',
          'soup':['genre_drama'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_romance':{
          'type':'Soup_just_1',
          'soup':['genre_romance'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_family':{
          'type':'Soup_just_1',
          'soup':['genre_family'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_crime':{
          'type':'Soup_just_1',
          'soup':['genre_crime'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_mystery':{
          'type':'Soup_just_1',
          'soup':['genre_mystery'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_war':{
          'type':'Soup_just_1',
          'soup':['genre_war'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_short':{
          'type':'Soup_just_1',
          'soup':['genre_short'],
          'skip_country': False,
          'include_in_next': False
      }

  }



  soft_hierarchy = {
      'group_GENRE_animation':{
          'type':'Soup_just_1',
          'soup':['genre_animation'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_war':{
          'type':'Soup_just_1',
          'soup':['genre_war'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_biography_OR_documentary_OR_history':{
          'type':'Soup_OR_3x1',
          'soup':[['genre_biography'],
                  ['genre_documentary'],
                  ['genre_history']],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_music_OR_music':{
          'type':'Soup_OR_2x1',
          'soup':[['genre_music'],
                  ['genre_musical']],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_history':{
          'type':'Soup_just_1',
          'soup':['genre_history'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_comedy':{
          'type':'Soup_just_1',
          'soup':['genre_comedy'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_drama_AND_family':{
          'type':'Soup_just_1',
          'soup':['genre_drama','genre_family'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_drama':{
          'type':'Soup_just_1',
          'soup':['genre_drama'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_adventure_or_fantasy':{
          'type':'Soup_OR_2x1',
          'soup':[['genre_adventure'],
                  ['genre_fantasy']],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_sport':{
          'type':'Soup_just_1',
          'soup':['genre_sport'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_romance':{
          'type':'Soup_just_1',
          'soup':['genre_romance'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_family ':{
          'type':'Soup_just_1',
          'soup':['genre_family'],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_scifi_OR_mystery':{
          'type':'Soup_OR_2x1',
          'soup':[['genre_sci-fi'],
                  ['genre_mystery']],
          'skip_country': False,
          'include_in_next': False
      },
      'group_GENRE_thriller_OR_action_OR_crime':{
          'type':'Soup_OR_3x2',
          'soup':[['genre_thriller'],
                  ['genre_action'],
                  ['genre_crime']],
          'skip_country': False,
          'include_in_next': False
      }
  }



  def __get_film_category(current_film_soup, current_keywords_by_space, hierarchy):


    for header in hierarchy:

      if hierarchy[header]['type'] == 'Soup_just_1':
        if set(hierarchy[header]['soup']).issubset(current_film_soup):
            return header

      elif hierarchy[header]['type'] == 'Soup_OR_2x2':
        if set(hierarchy[header]['soup'][0]).issubset(current_film_soup) or set(hierarchy[header]['soup'][1]).issubset(current_film_soup):
            return header
          
      elif hierarchy[header]['type'] == 'Soup_OR_2x1':
        if set(hierarchy[header]['soup'][0]).issubset(current_film_soup) or set(hierarchy[header]['soup'][1]).issubset(current_film_soup):
            return header

      elif hierarchy[header]['type'] == 'Soup_OR_3x2':
        if set(hierarchy[header]['soup'][0]).issubset(current_film_soup) or set(hierarchy[header]['soup'][1]).issubset(current_film_soup) or set(hierarchy[header]['soup'][2]).issubset(current_film_soup):
            return header

      elif hierarchy[header]['type'] == 'Soup_OR_3x1':
        if set(hierarchy[header]['soup'][0]).issubset(current_film_soup) or set(hierarchy[header]['soup'][1]).issubset(current_film_soup) or set(hierarchy[header]['soup'][2]).issubset(current_film_soup):
            return header

      elif hierarchy[header]['type'] == 'Soup_just_1_notEXACTLY_just_1':
        if set(hierarchy[header]['soup']).issubset(current_film_soup) and current_keywords_by_space.count(hierarchy[header]['not_exactly_tag'][0]) >= hierarchy[header]['times']:
            return header
          
      elif hierarchy[header]['type'] == 'genre_tags_or_11':
        if set(hierarchy[header]['soup'][0]).issubset(current_film_soup) or set(hierarchy[header]['soup'][1]).issubset(current_film_soup) or set(hierarchy[header]['soup'][2]).issubset(current_film_soup) \
        or set(hierarchy[header]['soup'][3]).issubset(current_film_soup) or set(hierarchy[header]['soup'][4]).issubset(current_film_soup) or set(hierarchy[header]['soup'][5]).issubset(current_film_soup) \
        or set(hierarchy[header]['soup'][6]).issubset(current_film_soup) or set(hierarchy[header]['soup'][7]).issubset(current_film_soup) or set(hierarchy[header]['soup'][8]).issubset(current_film_soup) \
        or set(hierarchy[header]['soup'][9]).issubset(current_film_soup) or set(hierarchy[header]['soup'][10]).issubset(current_film_soup):
            return header
          
      elif hierarchy[header]['type'] == 'genre_tags_or_21':
        if set(hierarchy[header]['soup'][0]).issubset(current_film_soup) or set(hierarchy[header]['soup'][1]).issubset(current_film_soup) or set(hierarchy[header]['soup'][2]).issubset(current_film_soup) \
        or set(hierarchy[header]['soup'][3]).issubset(current_film_soup) or set(hierarchy[header]['soup'][4]).issubset(current_film_soup) or set(hierarchy[header]['soup'][5]).issubset(current_film_soup) \
        or set(hierarchy[header]['soup'][6]).issubset(current_film_soup) or set(hierarchy[header]['soup'][7]).issubset(current_film_soup) or set(hierarchy[header]['soup'][8]).issubset(current_film_soup) \
        or set(hierarchy[header]['soup'][9]).issubset(current_film_soup) or set(hierarchy[header]['soup'][10]).issubset(current_film_soup) or set(hierarchy[header]['soup'][11]).issubset(current_film_soup) \
        or set(hierarchy[header]['soup'][12]).issubset(current_film_soup) or set(hierarchy[header]['soup'][13]).issubset(current_film_soup) or set(hierarchy[header]['soup'][14]).issubset(current_film_soup) \
        or set(hierarchy[header]['soup'][15]).issubset(current_film_soup) or set(hierarchy[header]['soup'][16]).issubset(current_film_soup) or set(hierarchy[header]['soup'][17]).issubset(current_film_soup) \
        or set(hierarchy[header]['soup'][18]).issubset(current_film_soup) or set(hierarchy[header]['soup'][19]).issubset(current_film_soup) or set(hierarchy[header]['soup'][20]).issubset(current_film_soup):
            return header
      
    return 'WRONG_FILM_PARAMETER'

    
  if 'russia' in current_country or 'ukraine' in current_country :
    header = __get_film_category(current_bag_of_words_for_train_nn, current_keywords_by_space, soft_hierarchy)
  else:
    header = __get_film_category(current_bag_of_words_for_train_nn, current_keywords_by_space, hard_hierarchy)


#   print('Header')
#   print(header, current_country)



  def __get_film_batch_for_category(df, header, hierarchy):

    batch = []

    exclude_headers = list(hierarchy.keys())[:list(hierarchy.keys()).index(header)]
    exclude_Soup_just_1 = []
    exclude_Soup_OR_2x2 = []
    exclude_Soup_OR_2x1 = []
    exclude_Soup_OR_3x2 = []
    exclude_Soup_OR_3x1 = []
    exclude_genre_tags_or_21 = []
    exclude_genre_tags_or_11 = []
    exclude_Soup_just_1_notEXACTLY_just_1 = []


    for excl_header in exclude_headers:
      if hierarchy[excl_header]['type'] == 'Soup_just_1' and hierarchy[excl_header]['include_in_next'] == False:
        exclude_Soup_just_1.append(hierarchy[excl_header]['soup'])

      elif hierarchy[excl_header]['type'] == 'Soup_OR_2x2' and hierarchy[excl_header]['include_in_next'] == False:
        exclude_Soup_OR_2x2.extend(hierarchy[excl_header]['soup'])
        
      elif hierarchy[excl_header]['type'] == 'Soup_OR_2x1' and hierarchy[excl_header]['include_in_next'] == False:
        exclude_Soup_OR_2x1.extend(hierarchy[excl_header]['soup'])

      elif hierarchy[excl_header]['type'] == 'Soup_OR_3x2' and hierarchy[excl_header]['include_in_next'] == False:
        exclude_Soup_OR_3x2.extend(hierarchy[excl_header]['soup'])

      elif hierarchy[excl_header]['type'] == 'Soup_OR_3x1' and hierarchy[excl_header]['include_in_next'] == False:
        exclude_Soup_OR_3x2.extend(hierarchy[excl_header]['soup'])
        
      elif hierarchy[excl_header]['type'] == 'genre_tags_or_11' and hierarchy[excl_header]['include_in_next'] == False:
        exclude_genre_tags_or_11.extend(hierarchy[excl_header]['soup'])
        
      elif hierarchy[excl_header]['type'] == 'genre_tags_or_21' and hierarchy[excl_header]['include_in_next'] == False:
        exclude_genre_tags_or_21.extend(hierarchy[excl_header]['soup'])

      elif hierarchy[excl_header]['type'] == 'Soup_just_1_notEXACTLY_just_1' and hierarchy[excl_header]['include_in_next'] == False:
        exclude_Soup_just_1_notEXACTLY_just_1.append([hierarchy[excl_header]['soup'], hierarchy[excl_header]['not_exactly_tag'], hierarchy[excl_header]['times']])


    if not exclude_Soup_just_1:
      exclude_Soup_just_1.append([None])

    if not exclude_Soup_OR_2x2:
      exclude_Soup_OR_2x2.extend([[None, None], [None, None]])
      
    if not exclude_Soup_OR_2x1:
      exclude_Soup_OR_2x1.extend([[None], [None]])

    if not exclude_Soup_OR_3x2:
      exclude_Soup_OR_3x2.extend([[None, None], [None, None], [None, None]])

    if not exclude_Soup_OR_3x2:
      exclude_Soup_OR_3x2.extend([[None], [None], [None]])
      
    if not exclude_genre_tags_or_11:
      exclude_genre_tags_or_11.extend([[None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None]])
      
    if not exclude_genre_tags_or_21:
      exclude_genre_tags_or_21.extend([[None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None]])

    if not exclude_Soup_just_1_notEXACTLY_just_1:
      exclude_Soup_just_1_notEXACTLY_just_1.append([[None], [None], -9999999999999999])


    for index, row in df.iterrows():


      soup = row['soup'].split(' ')
      imdb = row['imdbID']
      tags_not_exactly = row['keywords_by_space'].split(' ')

      flag = True
      for _exclude_Soup_just_1 in exclude_Soup_just_1:
        if set(_exclude_Soup_just_1).issubset(set(soup)):
          flag = False

      for _exclude_Soup_OR_2x2 in exclude_Soup_OR_2x2:
        if set(_exclude_Soup_OR_2x2).issubset(set(soup)):
          flag = False
       
      for _exclude_Soup_OR_2x1 in exclude_Soup_OR_2x1:
        if set(_exclude_Soup_OR_2x1).issubset(set(soup)):
          flag = False
       
      for _exclude_Soup_OR_3x2 in exclude_Soup_OR_3x2:
        if set(_exclude_Soup_OR_3x2).issubset(set(soup)):
          flag = False
          
      for _exclude_genre_tags_or_11 in exclude_genre_tags_or_11:
        if set(_exclude_genre_tags_or_11).issubset(set(soup)):
          flag = False
          
      for _exclude_genre_tags_or_21 in exclude_genre_tags_or_21:
        if set(_exclude_genre_tags_or_21).issubset(set(soup)):
          flag = False

      for _exclude_Soup_OR_3x1 in exclude_Soup_OR_3x1:
        if set(_exclude_Soup_OR_3x1).issubset(set(soup)):
          flag = False

      for _exclude_Soup_just_1_notEXACTLY_just_1 in exclude_Soup_just_1_notEXACTLY_just_1:
        if set(_exclude_Soup_just_1_notEXACTLY_just_1[0]).issubset(set(soup)) and tags_not_exactly.count(_exclude_Soup_just_1_notEXACTLY_just_1[1][0]) >= _exclude_Soup_just_1_notEXACTLY_just_1[2]:
          flag = False



      if flag:         
        if hierarchy[header]['type'] == 'Soup_just_1':
          if set(hierarchy[header]['soup']).issubset(set(soup)):
            batch.append(imdb)

        elif hierarchy[header]['type'] == 'Soup_OR_2x2':
          if set(hierarchy[header]['soup'][0]).issubset(set(soup)) or set(hierarchy[header]['soup'][1]).issubset(set(soup)):
            batch.append(imdb)
            
        elif hierarchy[header]['type'] == 'Soup_OR_2x1':
          if set(hierarchy[header]['soup'][0]).issubset(set(soup)) or set(hierarchy[header]['soup'][1]).issubset(set(soup)):
            batch.append(imdb)

        elif hierarchy[header]['type'] == 'Soup_OR_3x2':
          if set(hierarchy[header]['soup'][0]).issubset(set(soup)) or set(hierarchy[header]['soup'][1]).issubset(set(soup)) or set(hierarchy[header]['soup'][2]).issubset(set(soup)):
            batch.append(imdb)
            
        elif hierarchy[header]['type'] == 'genre_tags_or_11':
          if set(hierarchy[header]['soup'][0]).issubset(set(soup)) or set(hierarchy[header]['soup'][1]).issubset(set(soup)) or set(hierarchy[header]['soup'][2]).issubset(set(soup)) \
          or set(hierarchy[header]['soup'][3]).issubset(set(soup)) or set(hierarchy[header]['soup'][4]).issubset(set(soup)) or set(hierarchy[header]['soup'][5]).issubset(set(soup)) \
          or set(hierarchy[header]['soup'][6]).issubset(set(soup)) or set(hierarchy[header]['soup'][7]).issubset(set(soup)) or set(hierarchy[header]['soup'][8]).issubset(set(soup)) \
          or set(hierarchy[header]['soup'][9]).issubset(set(soup)) or set(hierarchy[header]['soup'][10]).issubset(set(soup)):
            batch.append(imdb)
            
        elif hierarchy[header]['type'] == 'genre_tags_or_21':
          if set(hierarchy[header]['soup'][0]).issubset(set(soup)) or set(hierarchy[header]['soup'][1]).issubset(set(soup)) or set(hierarchy[header]['soup'][2]).issubset(set(soup)) \
          or set(hierarchy[header]['soup'][3]).issubset(set(soup)) or set(hierarchy[header]['soup'][4]).issubset(set(soup)) or set(hierarchy[header]['soup'][5]).issubset(set(soup)) \
          or set(hierarchy[header]['soup'][6]).issubset(set(soup)) or set(hierarchy[header]['soup'][7]).issubset(set(soup)) or set(hierarchy[header]['soup'][8]).issubset(set(soup)) \
          or set(hierarchy[header]['soup'][9]).issubset(set(soup)) or set(hierarchy[header]['soup'][10]).issubset(set(soup)) or set(hierarchy[header]['soup'][11]).issubset(set(soup)) \
          or set(hierarchy[header]['soup'][12]).issubset(set(soup)) or set(hierarchy[header]['soup'][13]).issubset(set(soup)) or set(hierarchy[header]['soup'][14]).issubset(set(soup)) \
          or set(hierarchy[header]['soup'][15]).issubset(set(soup)) or set(hierarchy[header]['soup'][16]).issubset(set(soup)) or set(hierarchy[header]['soup'][17]).issubset(set(soup)) \
          or set(hierarchy[header]['soup'][18]).issubset(set(soup)) or set(hierarchy[header]['soup'][19]).issubset(set(soup)) or set(hierarchy[header]['soup'][20]).issubset(set(soup)):
            batch.append(imdb)

        elif hierarchy[header]['type'] == 'Soup_OR_3x1':
          if set(hierarchy[header]['soup'][0]).issubset(set(soup)) or set(hierarchy[header]['soup'][1]).issubset(set(soup)) or set(hierarchy[header]['soup'][2]).issubset(set(soup)):
            batch.append(imdb)

        elif hierarchy[header]['type'] == 'Soup_just_1_notEXACTLY_just_1':
          if set(hierarchy[header]['soup']).issubset(set(soup)) and tags_not_exactly.count(hierarchy[header]['not_exactly_tag'][0]) >= hierarchy[header]['times']:
            batch.append(imdb)


    batch = [film_index[i] for i in batch]
    return list(set(batch))
  if header != 'WRONG_FILM_PARAMETER':
    if 'russia' in current_country or 'ukraine' in current_country:
      country = 'Russia or Ukraine'
      batch = __get_film_batch_for_category(country_ukraine_russia_df, header, soft_hierarchy)
    else:
      if hard_hierarchy[header]['skip_country'] == False:
        if 'usa' in current_country:
          country ='USA'
          batch = __get_film_batch_for_category(country_usa_df, header, hard_hierarchy)
        else:
          country = 'EUROPE'
          batch = __get_film_batch_for_category(country_others_df, header, hard_hierarchy)
      else:
        country = 'ALL COUNTRIEs'
        batch = __get_film_batch_for_category(rated_df, header, hard_hierarchy)



  def __add_films(chunk, batch, closest_films, header):
    for i in closest_films:
      if i in batch and i not in [item[0] for item in chunk]:
        chunk.append([i , header])

    return chunk



  if header != 'WRONG_FILM_PARAMETER':
    chunk = __add_films(chunk, batch, closest_films, age_limit + ' ' + country + ' ' + header)


  ## ADDITIONAL FILMS ##
  if header != 'WRONG_FILM_PARAMETER':
    if 'russia' in current_country or 'ukraine' in current_country:

      if header == 'group_GENRE_animation':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation_and_TAG_disney', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_animation_and_TAG_disney')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation_and_TAG_minion', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_animation_and_TAG_minion')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_animation')


      elif header == 'group_GENRE_war':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_war', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_war')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_documentary_OR_history_OR_biography_AND_TAG_world_war_two', hard_hierarchy)
        chunk = __add_films(chunk, batch, country_not_ukr_rus_df, 'ADD_#2 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_documentary_OR_history_OR_biography_AND_TAG_world_war_two')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_documentary_war_OR_history_war_OR_biography_war', hard_hierarchy)
        chunk = __add_films(chunk, batch, country_not_ukr_rus_df, 'ADD_#2 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_documentary_war_OR_history_war_OR_biography_war')

        batch = __get_film_batch_for_category(country_ukraine_russia_df, 'group_GENRE_history', soft_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Russia or Ukraine' + ' ' + 'group_GENRE_history')

      elif header == 'group_GENRE_biography_OR_documentary_OR_history':  
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama')


      elif header == 'group_GENRE_music_OR_music':  
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_musical', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_musical')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_music', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_music')


      elif header == 'group_GENRE_history':  
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_history', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_history')

      elif header == 'group_GENRE_comedy':  
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_comedy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_comedy')

      elif header == 'group_GENRE_drama_AND_family':  
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_family_AND_TAG_animal', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_family_AND_TAG_animal')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_family', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_drama_AND_family')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_drama')

      elif header == 'group_GENRE_drama':  
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_drama')

      elif header == 'group_GENRE_adventure_or_fantasy':  
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_fantasy_AND_adventure', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_fantasy_AND_adventure')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_fantasy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_fantasy')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_adventure', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_adventure')


      elif header == 'group_GENRE_sport':  
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_sport', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_sport')


      elif header == 'group_GENRE_romance':  
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_romance', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_romance')


      elif header == 'group_GENRE_family':  
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_family', hard_hierarchy)
        chunk = __add_films(chunk, batch, country_not_ukr_rus_df, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_family')


      elif header == 'group_GENRE_scifi_OR_mystery':  
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_scifi', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_scifi')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_mystery', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_mystery')


      elif header == 'group_GENRE_thriller_OR_action_OR_crime':  
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_crime', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_crime')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_action_AND_thriller', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'Foreigner countries' + ' ' + 'group_GENRE_action_AND_thriller')

    #### OTHERS COUNTRIES ####    
    else:

      if header == 'group_GENRE_animation_and_TAG_minion':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation_and_TAG_minion', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation_and_TAG_minion')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation_and_TAG_disney', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation_and_TAG_disney')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_animation')

      elif header == 'group_GENRE_animation_and_TAG_racing':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation_and_TAG_racing', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation_and_TAG_racing')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation')

      elif header == 'group_GENRE_animation_and_TAG_superhero':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation_and_TAG_superhero', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation_and_TAG_superhero')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation_and_TAG_disney', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation_and_TAG_disney')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_animation')

      elif header == 'group_GENRE_animation_and_TAG_disney':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation_and_TAG_disney', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation_and_TAG_disney')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation')

      elif header == 'group_GENRE_animation_AND_musical_OR_animation_AND_music':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation_AND_musical_OR_animation_AND_music', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation_AND_musical_OR_animation_AND_music')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation')

      elif header == 'group_GENRE_animation_AND_TAG_dragon':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation_AND_TAG_dragon', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation_AND_TAG_dragon')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation')

      elif header == 'group_GENRE_animation_AND_comedy':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation_AND_comedy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation_AND_comedy')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation')

      elif header == 'group_GENRE_animation':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_animation_AND_comedy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_animation_AND_comedy')

      elif header == 'group_TAG_marvel_comic':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_TAG_marvel_comic', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_TAG_marvel_comic')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_TAG_dc_comic', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_TAG_dc_comic')

      elif header == 'group_TAG_dc_comic':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_TAG_dc_comic', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_TAG_dc_comic')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_TAG_marvel_comic', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_TAG_marvel_comic')

      elif header == 'group_TAG_based_on_comic_book':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_TAG_based_on_comic_book', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_TAG_based_on_comic_book')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_TAG_marvel_comic', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_TAG_marvel_comic')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_TAG_dc_comic', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_TAG_dc_comic') 

      elif header == 'group_GENRE_musical_and_TAG_dancing':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_musical_and_TAG_dancing', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_musical_and_TAG_dancing')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_music_TAG_dancing', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_music_TAG_dancing')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_musical', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_musical')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_music', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_music')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_biography_music_OR_history_music_OR_drama_music', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_biography_music_OR_history_music_OR_drama_music')

      elif header == 'group_GENRE_musical':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_musical', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_musical')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_musical_and_TAG_dancing', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_musical_and_TAG_dancing')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_music_TAG_dancing', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_music_TAG_dancing')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_music', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_music')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_biography_music_OR_history_music_OR_drama_music', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_biography_music_OR_history_music_OR_drama_music')

      elif header == 'group_GENRE_family_AND_TAG_animal':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_family_AND_TAG_animal', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_family_AND_TAG_animal')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_family', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_family')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_drama')

      elif header == 'group_GENRE_drama_AND_family':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_family', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_family')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_family', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_fantasy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_fantasy')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_adventure', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_adventure')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_adventure_AND_family_AND_fantasy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_adventure_AND_family_AND_fantasy')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_fantasy_AND_adventure', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_fantasy_AND_adventure')

      elif header == 'group_GENRE_comedy_and_TAGS_family_relationships':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_comedy_and_TAGS_family_relationships', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_comedy_and_TAGS_family_relationships')
        
      elif header == 'group_GENRE_comedy_and_TAGS_love_relationships':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_comedy_and_TAGS_love_relationships', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_comedy_and_TAGS_love_relationships')
        
        batch = __get_film_batch_for_category(rated_pg_13, 'group_GENRE_comedy_and_TAGS_love_relationships', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_comedy_and_TAGS_love_relationships')


      elif header == 'group_GENRE_music_TAG_dancing':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_music_TAG_dancing', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_music_TAG_dancing')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_musical_and_TAG_dancing', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_musical_and_TAG_dancing')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_music', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_music')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_musical', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_musical')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_biography_music_OR_history_music_OR_drama_music', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_biography_music_OR_history_music_OR_drama_music')

      elif header == 'group_GENRE_biography_music_OR_history_music_OR_drama_music':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_biography_music_OR_history_music_OR_drama_music', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_biography_music_OR_history_music_OR_drama_music')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_music', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_music')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_music_TAG_dancing', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_music_TAG_dancing')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_musical', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_musical')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_musical_and_TAG_dancing', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_musical_and_TAG_dancing')

      elif header == 'group_GENRE_music':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_music', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_music')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_music_TAG_dancing', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_music_TAG_dancing')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_musical', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_musical')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_musical_and_TAG_dancing', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_musical_and_TAG_dancing')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_biography_music_OR_history_music_OR_drama_music', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_biography_music_OR_history_music_OR_drama_music')
        

      elif header == 'group_GENRE_documentary_war_OR_history_war_OR_biography_war':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_documentary_war_OR_history_war_OR_biography_war', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_documentary_war_OR_history_war_OR_biography_war')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_war', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_war')

      elif header == 'group_GENRE_drama_AND_war':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_war', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_war')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_documentary_war_OR_history_war_OR_biography_war', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_documentary_war_OR_history_war_OR_biography_war')

      elif header == 'group_GENRE_documentary_OR_history_OR_biography_AND_TAG_world_war_two':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_documentary_OR_history_OR_biography_AND_TAG_world_war_two', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_documentary_OR_history_OR_biography_AND_TAG_world_war_two')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_war', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_war')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_documentary_war_OR_history_war_OR_biography_war', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_documentary_war_OR_history_war_OR_biography_war')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_war', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_war')

      elif header == 'group_GENRE_crime_AND_TAG_gangster':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_crime_AND_TAG_gangster', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_crime_AND_TAG_gangster')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_crime', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_crime')

      elif header == 'group_GENRE_sport_AND_comedy':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_sport_AND_comedy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_sport_AND_comedy')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_sport', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_sport')

      elif header == 'group_GENRE_sport':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_sport', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_sport')
        
        batch = __get_film_batch_for_category(rated_pg_13, 'group_GENRE_sport', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_sport')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_sport_AND_comedy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_sport_AND_comedy')
        
        batch = __get_film_batch_for_category(rated_pg_13, 'group_GENRE_sport_AND_comedy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_sport_AND_comedy')

      elif header == 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama')
                
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_history', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_history')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_biography', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_biography')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_documentary', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_documentary')

      elif header == 'group_GENRE_western':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_western', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_western')
        
        imdbids = country_not_ukr_rus_df[np.logical_and(np.logical_and(country_not_ukr_rus_df.soup.str.contains('shootout'), ~country_not_ukr_rus_df.soup.str.contains('genre_animation')),
                                                       np.logical_or(country_not_ukr_rus_df.soup.str.contains('genre_action'), country_not_ukr_rus_df.soup.str.contains('genre_thriller')))].imdbID.values

        batch = [film_index[_index] for _index in imdbids]
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_action_OR_thriller_and_TAG_shootout')

      elif header == 'group_GENRE_comedy_AND_horror':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_comedy_AND_horror', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_comedy_AND_horror')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_comedy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_comedy')

      elif header == 'group_GENRE_comedy_AND_TAG_zombie':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_comedy_AND_TAG_zombie', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_comedy_AND_TAG_zombie')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_comedy_AND_horror', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_comedy_AND_horror')

      elif header == 'group_GENRE_comedy_AND_TAG_kitchen':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_comedy_AND_TAG_kitchen', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_comedy_AND_TAG_kitchen')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_comedy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_comedy')

      elif header == 'group_GENRE_comedy_AND_drama':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_comedy_AND_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_comedy_AND_drama')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_comedy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_comedy')

      elif header == 'group_GENRE_comedy_AND_TAG_sex':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_comedy_AND_TAG_sex', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_comedy_AND_TAG_sex')

      elif header == 'group_GENRE_comedy':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_comedy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_comedy')

      elif header == 'group_GENRE_horror_OR_drama_AND_TAG_post_apocalypse':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_horror_OR_drama_AND_TAG_post_apocalypse', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_horror_OR_drama_AND_TAG_post_apocalypse')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_TAG_zombie', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_TAG_zombie')

      elif header == 'group_TAG_zombie':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_TAG_zombie', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_TAG_zombie')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_horror', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_horror')

      elif header == 'group_GENRE_horror_AND_mystery_AND_thriller':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_horror_AND_mystery_AND_thriller', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_horror_AND_mystery_AND_thriller')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_TAG_dc_comic', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_TAG_dc_comic')

      elif header == 'group_GENRE_drama_OR_thriller_AND_TAG_superhero':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_OR_thriller_AND_TAG_superhero', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama_OR_thriller_AND_TAG_superhero')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_thriller', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_thriller')

        
        
      elif header == 'group_GENRE_scifi_AND_TAG_female_protagonist':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_scifi_AND_TAG_female_protagonist', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_scifi_AND_TAG_female_protagonist')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_sci-fi_AND_TAG_martial_arts', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_sci-fi_AND_TAG_martial_arts')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_OR_thriller_AND_TAG_superhero', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_drama_OR_thriller_AND_TAG_superhero')
        
        
      elif header == 'group_GENRE_action_AND_TAG_female_protagonist':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_action_AND_TAG_female_protagonist', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_action_AND_TAG_female_protagonist')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_sci-fi_AND_TAG_martial_arts', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_sci-fi_AND_TAG_martial_arts')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_OR_thriller_AND_TAG_superhero', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_drama_OR_thriller_AND_TAG_superhero')
        
        

      elif header == 'group_GENRE_horror_AND_TAG_shark':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_horror_AND_TAG_shark', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_horror_AND_TAG_shark')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_thriller_AND_horror', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_thriller_AND_horror')
        
      elif header == 'group_GENRE_horror_AND_TAG_time_loop':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_horror_AND_TAG_time_loop', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_horror_AND_TAG_time_loop')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_thriller_AND_horror', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_thriller_AND_horror')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_horror', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_horror')
        

      elif header == 'group_GENRE_scifi_AND_horror':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_scifi_AND_horror', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_scifi_AND_horror')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_thriller_AND_horror', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_thriller_AND_horror')

      elif header == 'group_GENRE_sci-fi_AND_TAG_space_travel_OR_space':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_sci-fi_AND_TAG_space_travel_OR_space', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_sci-fi_AND_TAG_space_travel_OR_space')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_scifi', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_scifi')

      elif header == 'group_GENRE_sci-fi_AND_TAG_martial_arts':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_sci-fi_AND_TAG_martial_arts', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_sci-fi_AND_TAG_martial_arts')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_scifi_AND_TAG_female_protagonist', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_scifi_AND_TAG_female_protagonist')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_scifi', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_scifi')
        
        

      elif header == 'group_GENRE_sci-fi_AND_TAG_survivor':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_sci-fi_AND_TAG_survivor', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_sci-fi_AND_TAG_survivor')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_thriller', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_thriller')

      elif header == 'group_GENRE_thriller_AND_horror':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_thriller_AND_horror', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_thriller_AND_horror')
                
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_thriller', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_thriller')
        
        batch = __get_film_batch_for_category(rated_pg_13, 'group_GENRE_thriller_AND_horror', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_sport')
        
        batch = __get_film_batch_for_category(rated_pg_13, 'group_GENRE_thriller', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2' +  age_limit + ' ' + 'group_GENRE_sport')

      elif header == 'group_GENRE_action_AND_thriller':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_action_AND_thriller', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_action_AND_thriller')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_action', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_action')

      elif header == 'group_GENRE_adventure_AND_family_AND_fantasy':
        batch = __get_film_batch_for_category(rated_df, 'group_GENRE_adventure_AND_family_AND_fantasy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_adventure_AND_family_AND_fantasy')
        
        batch = __get_film_batch_for_category(rated_df, 'group_GENRE_fantasy_AND_adventure', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_fantasy_AND_adventure')
                
        batch = __get_film_batch_for_category(rated_df, 'group_GENRE_fantasy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_fantasy')
        
        batch = __get_film_batch_for_category(rated_df, 'group_GENRE_family', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_family')
        
        batch = __get_film_batch_for_category(rated_df, 'group_GENRE_drama_AND_family', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_family')
        
        batch = __get_film_batch_for_category(rated_df, 'group_GENRE_adventure', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_adventure')
        

      elif header == 'group_GENRE_scifi':
        batch = __get_film_batch_for_category(rated_df, 'group_GENRE_scifi', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_scifi')

      elif header == 'group_GENRE_biography':        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_biography', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_biography')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_history', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_history')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_documentary', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_documentary')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama')
        
        batch = __get_film_batch_for_category(rated_pg_13, 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama')

      elif header == 'group_GENRE_history':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_history', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_history')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_biography', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_biography')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_documentary', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_documentary')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama')
        
        batch = __get_film_batch_for_category(rated_pg_13, 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama')

      elif header == 'group_GENRE_documentary':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_documentary', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_documentary')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_history', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_history')

        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_biography', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_biography')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama')
        
        batch = __get_film_batch_for_category(rated_pg_13, 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_biography_drama_OR_history_drama_OR_documentary_drama')

      elif header == 'group_GENRE_alien':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_alien', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_alien')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_scifi', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_scifi')

      elif header == 'group_GENRE_drama_AND_romance':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_romance', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_romance')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_and_TAGS_love_relationships', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama_and_TAGS_love_relationships')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_romance', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_romance')
        

      elif header == 'group_GENRE_drama_AND_thriller':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_thriller', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_thriller')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_thriller', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_thriller')

      elif header == 'group_GENRE_fantasy_AND_adventure':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_fantasy_AND_adventure', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_fantasy_AND_adventure')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_fantasy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_adventure')

      elif header == 'group_GENRE_fantasy':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_fantasy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_fantasy')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_fantasy_AND_adventure', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_fantasy_AND_adventure')

      elif header == 'group_GENRE_action':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_action', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_action')

      elif header == 'group_GENRE_adventure':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_adventure', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_adventure')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_fantasy_AND_adventure', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_fantasy_AND_adventure')

      elif header == 'group_GENRE_horror':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_horror', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_horror')

      elif header == 'group_GENRE_thriller':
        batch = __get_film_batch_for_category(rated_df, 'group_GENRE_thriller', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_thriller')
        
      
      elif header == 'group_GENRE_drama_and_TAGS_love_relationships':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_and_TAGS_love_relationships', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama_and_TAGS_love_relationships')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_romance', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_romance')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_romance', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_romance')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_family', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_family')
        
      elif header == 'group_GENRE_drama_and_TAGS_family_relationships':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_and_TAGS_family_relationships', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama_and_TAGS_family_relationships')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_romance', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_romance')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_romance', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_romance')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_family', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_family')
    
      elif header == 'group_GENRE_drama':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_drama')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_family', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_family')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_and_TAGS_family_relationships', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_drama_and_TAGS_family_relationships')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_romance', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_romance')

      elif header == 'group_GENRE_romance':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_romance', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_romance')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_romance', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_romance')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_and_TAGS_love_relationships', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_drama_and_TAGS_love_relationships')

      elif header == 'group_GENRE_family':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_family', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_family')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_drama_AND_family', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_drama_AND_family')
        
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_comedy', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#2 ' +  age_limit + ' ' + 'group_GENRE_comedy')

      elif header == 'group_GENRE_crime':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_crime', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_crime')

      elif header == 'group_GENRE_mystery':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_mystery', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_mystery')

      elif header == 'group_GENRE_war':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_war', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_war')

      elif header == 'group_GENRE_short':
        batch = __get_film_batch_for_category(country_not_ukr_rus_df, 'group_GENRE_short', hard_hierarchy)
        chunk = __add_films(chunk, batch, closest_films, 'ADD_#1 ' +  age_limit + ' ' + 'group_GENRE_short')


  if header != 'WRONG_FILM_PARAMETER':
    return chunk
  else:
    return 'WRONG_FILM_PARAMETER'


  

In [0]:
def main(path_to_db_films, output_path, model_name='default', save_model=True, path_to_model='your_path', debug = True, IMDB_ID='tt5884230'):
  df = pd.read_csv(path_to_your_dir + path_to_db_films)
#   df.loc[df[df.long == '00000000000000000000000000001361'].index, 'genre'] = 'genre_music genre_musical genre_comedy genre_drama'
#   df.loc[df[df.long == '00000000000000000000000000000846'].index, 'soup'] = df[df.long == '00000000000000000000000000000846'].soup.str.replace('gangster ', '', n=1).values[0]

  df.fillna('', inplace=True)
#   df = df[df.genre != ''].reset_index(drop=True)
#   df = df.drop_duplicates('imdbID')
  df.loc[df[np.logical_and(df.genre.str.contains('genre_animation'), df.rated == '')].index, 'rated'] = 'PG'

  # Складки часу і Tommorowland рейтед міняю
  df.loc[df[df.long == '00000000000000000000000000000759'].index, 'rated'] = 'PG-13'
  df.loc[df[df.long == 'f744954c96a9758b13c048f7371561b3'].index, 'rated'] = 'PG-13'
  
  


  pairs, pairs_set, tag_index, film_index, index_film, tags, films = make_pairs_for_train_NN(df)


  if save_model:
    model = train_model(df, pairs, pairs_set, tag_index, film_index, index_film, tags, films, epoch=200, batch_size=256)
    model.save(path_to_your_dir + model_name + '.h5') 
  else:
    model = load_model(path_to_your_dir + model_name + '.h5')
  
  
  if debug:
    closest_films, distances = get_similar_film_by_nn(df[df.imdbID == IMDB_ID].imdbID.values[0], model, film_index, index_film)
    distances = np.interp(distances, (distances.min(), distances.max()), (0, +0.98))    
    zip_film_score = list(zip(closest_films, distances))
    
    
    chunk = get_chunk(df, IMDB_ID, closest_films, film_index)
    
    if chunk != 'WRONG_FILM_PARAMETER':
      chunk = pd.DataFrame(chunk, columns=['indexx', 'Category'])
      zip_film_score = pd.DataFrame(zip_film_score, columns=['indexx', 'Coef'])

      result = pd.merge(chunk, zip_film_score, how='inner', on=['indexx'])

      result['indexx'] = [index_film[i] for i in result['indexx'].tolist()]

      res_title = [df[df.imdbID == i].originalTitle.values[0] for i in result['indexx'].tolist()]

      result['res_title'] = res_title
      
      result.loc[result[result.Category.str.contains('ADD_#1')]['Coef'].index, 'Coef'] = result[result.Category.str.contains('ADD_#1')]['Coef'] / 7 
      result.loc[result[result.Category.str.contains('ADD_#2')]['Coef'].index, 'Coef'] = result[result.Category.str.contains('ADD_#2')]['Coef'] / 14
      result.loc[result[result.Category.str.contains('director')]['Coef'].index, 'Coef'] = 0.99
      
      result['Coef'] = result['Coef'].round(2) * 100
      result['Coef'] = result['Coef'].astype(int)

      return result[['res_title', 'Category', 'Coef']]    
    else:
      return 'WRONG_FILM_PARAMETER'
    
  else:
    
    result = pd.DataFrame()
    
    for index, row in tqdm(df.iterrows()):
      
      closest_films, distances = get_similar_film_by_nn(row['imdbID'], model, film_index, index_film)
      distances = np.interp(distances, (distances.min(), distances.max()), (0, +0.98))
      zip_film_score = list(zip(closest_films, distances))

      chunk = get_chunk(df, row['imdbID'], closest_films, film_index)
      
      if chunk != 'WRONG_FILM_PARAMETER':
        chunk = pd.DataFrame(chunk, columns=['indexx', 'Category'])
        zip_film_score = pd.DataFrame(zip_film_score, columns=['indexx', 'Coef'])


        temp = pd.merge(chunk, zip_film_score, how='inner', on=['indexx'])

        temp['indexx'] = [index_film[i] for i in temp['indexx'].tolist()]
        lon_rec = [df[df.imdbID == i].long.values[0] for i in temp['indexx'].tolist()]
        temp['long_orig'] = df[df.imdbID == row['imdbID']].long.values[0]
        temp['long_rec'] = lon_rec

        temp = temp[:20]
        
        temp.loc[temp[temp.Category.str.contains('ADD_#1')]['Coef'].index, 'Coef'] = temp[temp.Category.str.contains('ADD_#1')]['Coef'] / 7 
        temp.loc[temp[temp.Category.str.contains('ADD_#2')]['Coef'].index, 'Coef'] = temp[temp.Category.str.contains('ADD_#2')]['Coef'] / 14
        temp.loc[temp[temp.Category.str.contains('director')]['Coef'].index, 'Coef'] = 0.99
        
        temp['Coef'] = temp['Coef'].round(2) * 100
        temp['Coef'] = temp['Coef'].astype(int)
        
        result = result.append(temp[['long_orig', 'long_rec', 'Coef', 'Category']])
        
        
      else:
        print(row['imdbID'])
        print('WRONG_FILM_PARAMETER')
      
    result.to_excel(path_to_your_dir + output_path, index=False)
      
      
  

In [36]:
main('prod42.csv', 'all_films_21_05_1_20films.xlsx', model_name='epoch_200', save_model = True, debug = False, IMDB_ID='tt0816692')

Epoch 1/200
399/399 [==============================] - 4s 9ms/step - loss: 0.9563
Epoch 2/200
399/399 [==============================] - 3s 9ms/step - loss: 0.6486
Epoch 3/200
399/399 [==============================] - 3s 9ms/step - loss: 0.4786
Epoch 4/200
399/399 [==============================] - 3s 8ms/step - loss: 0.4510
Epoch 5/200
399/399 [==============================] - 3s 9ms/step - loss: 0.4386
Epoch 6/200
399/399 [==============================] - 3s 8ms/step - loss: 0.4323
Epoch 7/200
399/399 [==============================] - 3s 8ms/step - loss: 0.4248
Epoch 8/200
399/399 [==============================] - 3s 9ms/step - loss: 0.4248
Epoch 9/200
399/399 [==============================] - 3s 8ms/step - loss: 0.4210
Epoch 10/200
399/399 [==============================] - 4s 9ms/step - loss: 0.4197
Epoch 11/200
399/399 [==============================] - 4s 9ms/step - loss: 0.4208
Epoch 12/200
399/399 [==============================] - 3s 9ms/step - loss: 0.4195
Epoch 13/200


In [0]:
""